In [1]:
# Project Overview: Create Relevant Spotify Playlist with Python
# Pre-work: Set up your Account & Register your application @ https://developer.spotify.com/documentation/web-api/quick-start/
# pip install requirements.txt
# 0. Declare name of playlist to be created
# 1. Import Packages and establish creds to connect to Spotify
# 2. Get raw Spotify data of Liked (Hearted) songs into a dataframe
# 3. Add audio features
# 4. Add genre data
# 5. Create genre score
### Repeat process (steps 2-5) for disliked songs (based on dislike playlist) ###
# 6. Create playlist df (to access disliked playlist)
# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
# 8. Get featured playlist uris so we can get their associated songs
# 9. Get raw data from the songs
# ....
# 10. Create playlist

In [2]:
# 0. Declare name of playlist to be created
pred_like_playlist_name = "90s"

In [9]:
#### ALL FUNCTIONS BELOW ####

# 1. Import Packages and establish creds to connect to Spotify
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth

from config import CONFIG
from datetime import datetime, timedelta
import inspect
import io
import json
from IPython.display import display
from json.decoder import JSONDecodeError
import math
import os
import pandas as pd
import numpy as np 
from pandas.io.json import json_normalize
import pickle
import requests
import subprocess
import sys
import time
import webbrowser

API_BASE = 'https://accounts.spotify.com'
CLI_ID = CONFIG["env_variables"]['SPOTIPY_CLIENT_ID']
CLI_SEC = CONFIG["env_variables"]['SPOTIPY_CLIENT_SECRET']
REDIRECT_URI = CONFIG["env_variables"]['SPOTIPY_REDIRECT_URI']
SHOW_DIALOG = False
SCOPE = '''
    playlist-modify-private, 
    playlist-modify-public,
    playlist-read-collaborative,
    playlist-read-private,
    user-follow-modify,
    user-follow-read,
    user-library-modify,
    user-library-read,
    user-modify-playback-state, 
    user-read-currently-playing, 
    user-read-email,
    user-read-playback-state, 
    user-read-private,
    user-read-recently-played, 
    user-top-read'''

def req(url,headers):
    r = requests.get(url, headers=headers)
    return json.loads(r.text)

#1. get raw data for liked songs (i.e. hearted)
def get_raw_liked_song_list():
    print('Get_raw_liked_song_list')
    offset = 0
    url = f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}"

    liked_song_count = req(url,headers=headers)["total"]
    print(f"Liked song count: {liked_song_count}")

    raw_liked_songs = []
    for i in range(int(math.ceil(liked_song_count/50.0))):
        offset = 50*i
        raw_liked_songs.extend(req(f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}",headers=headers)["items"])

    print("Example of song raw data from spotify:")
    print(f"{raw_liked_songs[0]}")
    return raw_liked_songs

# 2b. transform liked song raw data list iddnto df
# parse out data from Liked Songs' raw data
def song_metadata_to_df(raw_songs):
    print('Transforming song raw data list into DataFrame(df) - this is basically a python version of an excel spreadsheet')
    define_scope()
    # create empty df
    df = pd.DataFrame(columns = [
        'album', 
        'artist_uri', 
        'artist',
        'track',
        'url',
        'popularity',
        'added_at',
        'release_date',
        'release_year',
        'uri'
        ])
    # parse out songs 100 at a time
    for i in range(0, len(raw_songs)):
        if (i/100).is_integer():
            print(f"{i}/{len(raw_songs)} parsed")
        try:
            album = raw_songs[i]['track']['album']['name']
            artist_uri = raw_songs[i]['track']['artists'][0]['uri'].split(':')[2]
            artist = raw_songs[i]['track']['album']['artists'][0]['name']
            track = raw_songs[i]['track']['name']
            url = raw_songs[i]['track']['external_urls']['spotify']
            popularity = raw_songs[i]['track']['popularity']
            added_at = raw_songs[i]['added_at']
            release_date = raw_songs[i]['track']['album']['release_date']
            release_year = release_date.split('-')[0]
            uri = raw_songs[i]['track']['uri']
            df = df.append(pd.DataFrame([[
                album, 
                artist_uri,
                artist, 
                track, 
                url, 
                popularity, 
                added_at, 
                release_date, 
                release_year,
                uri
                ]], columns = [
                'album', 
                'artist_uri', 
                'artist',
                'track',
                'url',
                'popularity',
                'added_at',
                'release_date',
                'release_year',
                'uri'
                ]))
        except:
            pass
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

#3. Add audio features 
# Combine liked song df with audio feature df
def add_audio_feats(df):
    print('Combining song_df with audio_feature_df')
    # get list of song uris
    song_uri_list = df.index.to_list()
    audio_feature_df = pd.DataFrame()
    for i in range(0,math.ceil(len(song_uri_list)/100)):
        features = sp.audio_features(song_uri_list[(i)*100:100*(i+1)])
        audio_feature_df = audio_feature_df.append(pd.DataFrame(features))
    audio_feature_df.set_index('uri', inplace=True)
    # merge this 
    liked_song_df = df.merge(audio_feature_df, on = 'uri')
    display(liked_song_df.head())
    return liked_song_df.drop_duplicates()

#4. Add genre data
# 4a. Get artist_uris -> get genres
def get_genres(df):
    print('Getting genres ...')
    g = []
    artist_uris = df['artist_uri'].unique()
    print(f"{len(artist_uris)} distinct artists")
    for i in range(0, len(artist_uris)):
        if (i/100).is_integer():
            print(f"{i}/{len(artist_uris)} parsed")
        request = req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)
        artist = request['name']
        genres = request['genres']
        print(f"{i}:{artist}:{genres}")
        g.append(genres)
#         socketio.sleep(0.1)

    genres_df = pd.DataFrame({'artist_uri': artist_uris, 'genre list': g})
    display(genres_df.head())
    return genres_df

# There is a list of genres for one artist that must be broken out
def explode_genres(df):
    genre_exploded_df = df.explode('genre list')
    
    print('genre_exploded_df')
    display(genre_exploded_df.head())
    return genre_exploded_df

# Add these genres back to the df for each song
def add_genres(df, genre_exploded_df):
    # the following does not work oddly enough
    # liked_song_df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri', inplace = True)
    df = df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri')
    df.rename(columns={'genre list': 'genre'}, inplace=True)
    
    print('Merging genres into song_df')
    display(df.head())
    return df

#5. Create Genre Score
# Get genre counts for liked songs - the higher the count, the higher the score
def get_genre_counts(genre_exploded_df):
    print('Getting genre counts')
    # establish genre counts (which will determine score)
    genre_count_df = pd.DataFrame(genre_exploded_df['genre list'].value_counts()).reset_index()
    genre_count_df.columns = ['genre','genre count']
    print('genre_count_df.head()')
    display(genre_count_df.head())
    return genre_count_df

# Merge the genre counts with the song df and sum the genre column
def establish_genre_score(genre_count_df, df):
    print('Merging genre counts with song_df')
    df = df.reset_index().merge(genre_count_df , on = 'genre',how = 'left')
    df.drop(columns=['genre'], inplace=True)
    display(df.head())

    groupby_cols = df.columns.to_list()[:-1]
    
    print('Summing genre column')
    df = df.groupby(groupby_cols).sum().sort_values('genre count', ascending=False)
    df = df.rename({'genre count': 'genre score'}, axis=1).reset_index() 
    # reset index after summing - otherwise everything prior to sum will become index
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

# 6. Create playlist df (to access disliked playlist)
# get playlist raw data (for dislikes)
def get_playlist_uris(playlists_raw):
    df = pd.DataFrame()
    for i in range(0, len(playlists_raw)):
        try:
            playlist = playlists_raw[i]['name']
        except:
            playlist = ''
        try:
            playlist_uri = playlists_raw[i]['uri']
        except:
            playlist_uri = ''
        try:
            song_count = playlists_raw[i]['tracks']['total']
        except:
            song_count = 0
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    display(df)
    return df

def get_raw_disliked_song_list():
    playlists_raw = sp.current_user_playlists()['items']
    playlists_df = get_playlist_uris(playlists_raw)
    display(playlists_df)
    uri = playlists_df.loc['Dislikes']['playlist_uri']
    song_count = playlists_df.loc['Dislikes']['song_count']
    # extract data from Disliked songs
    dislike_raw_list = []
    for i in range(int(math.ceil(song_count/100.0))):
        dislike_raw_list.extend(sp.user_playlist_tracks(uri.split(':')[1], uri.split(':')[2], offset=i*100)['items'])
    print('Getting raw disliked song list')
    return dislike_raw_list

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
def combine_dfs_to_create_my_songs_df(df1,df2, save_destination):
    print('Combining dfs to create my_songs_df')
    df1['Liked'] = 1
    df2['Liked'] = 0
    my_songs_df = pd.concat([df1,df2])
    #my_songs_df.to_:w(f"{save_destination}/my_songs.csv")
    #my_songs_df.to_pickle(f"{save_destination}/my_songs.pkl")
    display(my_songs_df.head())
    #print('successfully saved to .csv and .pkl files in save_destination')
    return 'OK'

# 8. Get featured playlist uris so we can get their associated songs
def get_featured_playlist_uris(country, timestamp):
    print('Getting raw_featured_playlist_df')
    url = f"https://api.spotify.com/v1/browse/featured-playlists?limit=50&offset=0&country={country}&timestamp={timestamp}"
    featured_playlist_count = req(url,headers=headers)['playlists']['total']
    df = pd.DataFrame()
    for i in range(0,featured_playlist_count):
        raw_featured_playlist = req(url,headers=headers)['playlists']['items']
        playlist = raw_featured_playlist[i]['name']
        playlist_uri = raw_featured_playlist[i]['uri']
        song_count = raw_featured_playlist[i]['tracks']['total']
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    display(df)
    return df

# 9. Get raw data from the songs
def get_raw_featured_playlist_song_list(df):
    raw_featured_songs = []
    for uri in df.loc[:,'playlist_uri'].to_list():
        song_count = df.loc[df['playlist_uri']==uri,'song_count'][0]
        offset = 0
        url = f"https://api.spotify.com/v1/playlists/{uri.split(':')[2]}/tracks?limit=50&offset={offset}"
        for i in range(int(math.ceil(song_count/100.0))):
            offset = 100*i
            raw_featured_songs.extend(req(url,headers=headers)['items'])

    return raw_featured_songs

def create_playlist(pred_like_playlist_name, df):
    # create playlist
    define_scope()
    payload = {"name": pred_like_playlist_name}
    result = requests.post(url = f'https://api.spotify.com/v1/users/{username}/playlists'
                    , headers = headers
                    , data = json.dumps(payload)
                    )
    playlists_raw = sp.current_user_playlists()['items']
    playlists_df = get_playlist_uris(playlists_raw)
    playlist_uri = get_playlist_uris(playlists_raw).loc[playlists_df.index == pred_like_playlist_name,'playlist_uri'][0].split(':')[2]
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rank.html
    df['pct_rank'] = df['genre score'].rank(pct=True)
    df = df.drop_duplicates()
    display(df.head())
    # print((1-(1-1/((int(genre_score_threshold)/50)+1.1))))
    uri_list = df.loc[df['pct_rank'] >= 0.95].reset_index()['uri'].to_list()
    num_songs_to_add = len(uri_list)
    # add songs
    for i in range(int(math.ceil(num_songs_to_add/100.0))):
        url = f"https://api.spotify.com/v1/playlists/{playlist_uri}/tracks?uris={','.join(uri_list[i*100:(i+1)*100])}"
        requests.post(url, headers=headers)
    return 'OK'

# def verify():
#     global auth_url
#     auth_url = f'{API_BASE}/authorize?client_id={CLI_ID}&response_type=code&redirect_uri={REDIRECT_URI}&scope={SCOPE}&show_dialog={SHOW_DIALOG}'
#     return auth_url

# def api_callback():
#     global token
# #     session.clear()
#     code = request.args.get('code')

#     auth_token_url = f"{API_BASE}/api/token"
#     res = requests.post(auth_token_url, data={
#         "grant_type":"authorization_code",
#         "code":code,
#         "redirect_uri":REDIRECT_URI,
#         "client_id":CLI_ID,
#         "client_secret":CLI_SEC
#         })

#     res_body = res.json()
#     print(res.json())
#     token = res_body.get("access_token")
#     return token

def define_scope():
    global scope, sp, user, username, headers
    scope = '''
    playlist-modify-private 
    playlist-modify-public 
    playlist-read-collaborative 
    playlist-read-private
    user-follow-modify 
    user-follow-read 
    user-library-modify 
    user-library-read 
    user-modify-playback-state 
    user-read-currently-playing 
    user-read-email 
    user-read-playback-state 
    user-read-private 
    user-read-recently-played 
    user-top-read'''
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    user = sp.current_user()
    username = user['display_name']
    token = util.prompt_for_user_token(username, scope)
    headers = {'Authorization': "Bearer {}".format(token)}
    print(f"Logged in as {user['display_name']}")
    return

In [10]:
define_scope()
print(f"Your new playlist: {pred_like_playlist_name}")
# print(f"genre threshold = {genre_threshold}")
# print(f"featured playlist year = {featured_playlist_year} ({year_delta} years ago)")

print('Scope defined')
print('')
print('****PART 1 of 4: Get Liked Songs Dataframe****')
raw_liked_song_list = get_raw_liked_song_list()
liked_song_df = song_metadata_to_df(raw_liked_song_list)
liked_song_df = add_audio_feats(liked_song_df)
genre_df = get_genres(liked_song_df)
genre_exploded_df = explode_genres(genre_df)
liked_song_df = add_genres(liked_song_df, genre_exploded_df)
liked_genre_count_df = get_genre_counts(genre_exploded_df)
liked_song_df = establish_genre_score(liked_genre_count_df, liked_song_df)

Logged in as 1254636534
Your new playlist: 90s
Scope defined

****PART 1 of 4: Get Liked Songs Dataframe****
Get_raw_liked_song_list
Liked song count: 629
Example of song raw data from spotify:
{'added_at': '2020-09-25T13:10:04Z', 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7B2edU3Q7btJoNsoHCNohM'}, 'href': 'https://api.spotify.com/v1/artists/7B2edU3Q7btJoNsoHCNohM', 'id': '7B2edU3Q7btJoNsoHCNohM', 'name': 'Laura Marling', 'type': 'artist', 'uri': 'spotify:artist:7B2edU3Q7btJoNsoHCNohM'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'KZ', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', '

,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:1vlDwedZUvYaxt2FXoYGaJ,Song For Our Daughter,7B2edU3Q7btJoNsoHCNohM,Laura Marling,Held Down,https://open.spotify.com/track/1vlDwedZUvYaxt2...,56,2020-09-25T13:10:04Z,2020-04-10,2020
spotify:track:7i8MdTMo8ZibQhvhqzWhIJ,I Need a Teacher,37eqxl8DyLd5sQN54wYJbE,Hiss Golden Messenger,I Need a Teacher,https://open.spotify.com/track/7i8MdTMo8ZibQhv...,0,2020-09-25T12:55:58Z,2019-06-11,2019
spotify:track:7kegj615xcwQgwfhzCdIFE,It Doesn't Really Feel Like Spring,0syIRg9MPSpJIC0QCPUaHB,Beta Radio,It Doesn't Really Feel Like Spring,https://open.spotify.com/track/7kegj615xcwQgwf...,39,2020-09-22T17:21:39Z,2020-07-03,2020
spotify:track:04bAqpyuPCsgZAfnbsGDXx,Water (FKJ Remix),6Tyzp9KzpiZ04DABQoedps,Little Dragon,Water - FKJ Remix,https://open.spotify.com/track/04bAqpyuPCsgZAf...,46,2020-09-18T20:19:37Z,2020-07-28,2020
spotify:track:7eSWgAlmLPXbGxgH3RNoWh,We Stayed Up All Night,2ABBMkcUeM9hdpimo86mo6,Tourist,We Stayed Up All Night,https://open.spotify.com/track/7eSWgAlmLPXbGxg...,63,2020-09-17T15:54:24Z,2017-06-28,2017


Combining song_df with audio_feature_df


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:1vlDwedZUvYaxt2FXoYGaJ,Song For Our Daughter,7B2edU3Q7btJoNsoHCNohM,Laura Marling,Held Down,https://open.spotify.com/track/1vlDwedZUvYaxt2...,56,2020-09-25T13:10:04Z,2020-04-10,2020,0.488,...,0.000002,0.3530,0.399,79.442,audio_features,1vlDwedZUvYaxt2FXoYGaJ,https://api.spotify.com/v1/tracks/1vlDwedZUvYa...,https://api.spotify.com/v1/audio-analysis/1vlD...,247920,4
spotify:track:7i8MdTMo8ZibQhvhqzWhIJ,I Need a Teacher,37eqxl8DyLd5sQN54wYJbE,Hiss Golden Messenger,I Need a Teacher,https://open.spotify.com/track/7i8MdTMo8ZibQhv...,0,2020-09-25T12:55:58Z,2019-06-11,2019,0.499,...,0.001330,0.2870,0.678,85.985,audio_features,7i8MdTMo8ZibQhvhqzWhIJ,https://api.spotify.com/v1/tracks/7i8MdTMo8Zib...,https://api.spotify.com/v1/audio-analysis/7i8M...,197597,4
spotify:track:7kegj615xcwQgwfhzCdIFE,It Doesn't Really Feel Like Spring,0syIRg9MPSpJIC0QCPUaHB,Beta Radio,It Doesn't Really Feel Like Spring,https://open.spotify.com/track/7kegj615xcwQgwf...,39,2020-09-22T17:21:39Z,2020-07-03,2020,0.558,...,0.571000,0.0734,0.165,95.018,audio_features,7kegj615xcwQgwfhzCdIFE,https://api.spotify.com/v1/tracks/7kegj615xcwQ...,https://api.spotify.com/v1/audio-analysis/7keg...,320670,4
spotify:track:04bAqpyuPCsgZAfnbsGDXx,Water (FKJ Remix),6Tyzp9KzpiZ04DABQoedps,Little Dragon,Water - FKJ Remix,https://open.spotify.com/track/04bAqpyuPCsgZAf...,46,2020-09-18T20:19:37Z,2020-07-28,2020,0.792,...,0.005320,0.1040,0.415,113.503,audio_features,04bAqpyuPCsgZAfnbsGDXx,https://api.spotify.com/v1/tracks/04bAqpyuPCsg...,https://api.spotify.com/v1/audio-analysis/04bA...,199103,5
spotify:track:7eSWgAlmLPXbGxgH3RNoWh,We Stayed Up All Night,2ABBMkcUeM9hdpimo86mo6,Tourist,We Stayed Up All Night,https://open.spotify.com/track/7eSWgAlmLPXbGxg...,63,2020-09-17T15:54:24Z,2017-06-28,2017,0.519,...,0.000446,0.2100,0.592,84.997,audio_features,7eSWgAlmLPXbGxgH3RNoWh,https://api.spotify.com/v1/tracks/7eSWgAlmLPXb...,https://api.spotify.com/v1/audio-analysis/7eSW...,265759,4


Getting genres ...
403 distinct artists
0/403 parsed
0:Laura Marling:['art pop', 'british folk', 'british singer-songwriter', 'chamber pop', 'indie folk', 'indie pop', 'singer-songwriter', 'stomp and holler']
1:Hiss Golden Messenger:['alternative country', 'deep new americana', 'indie folk', 'indie rock', 'modern folk rock', 'new americana', 'north carolina indie', 'roots rock', 'stomp and holler']
2:Beta Radio:['acoustic pop', 'indie anthem-folk', 'indie folk', 'indiecoustica', 'new americana', 'stomp and flutter', 'stomp and holler']
3:Little Dragon:['alternative dance', 'alternative r&b', 'art pop', 'chillwave', 'downtempo', 'electronica', 'electropop', 'escape room', 'gothenburg indie', 'indie soul', 'indietronica', 'new rave', 'swedish electropop', 'swedish synthpop', 'trip hop']
4:Tourist:['chillwave', 'electronica', 'indie r&b', 'indie soul', 'shimmer pop', 'tropical house', 'vapor soul']
5:Tunng:['folktronica']
6:Caamp:['indie folk', 'new americana', 'stomp and holler']
7:Ryan 

...

,genre,genre count
0,indie pop,74
1,indie rock,73
2,stomp and holler,72
3,indie folk,63
4,modern rock,52


Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre count
0,spotify:track:1vlDwedZUvYaxt2FXoYGaJ,Song For Our Daughter,7B2edU3Q7btJoNsoHCNohM,Laura Marling,Held Down,https://open.spotify.com/track/1vlDwedZUvYaxt2...,56,2020-09-25T13:10:04Z,2020-04-10,2020,...,0.353,0.399,79.442,audio_features,1vlDwedZUvYaxt2FXoYGaJ,https://api.spotify.com/v1/tracks/1vlDwedZUvYa...,https://api.spotify.com/v1/audio-analysis/1vlD...,247920,4,29.0
1,spotify:track:1vlDwedZUvYaxt2FXoYGaJ,Song For Our Daughter,7B2edU3Q7btJoNsoHCNohM,Laura Marling,Held Down,https://open.spotify.com/track/1vlDwedZUvYaxt2...,56,2020-09-25T13:10:04Z,2020-04-10,2020,...,0.353,0.399,79.442,audio_features,1vlDwedZUvYaxt2FXoYGaJ,https://api.spotify.com/v1/tracks/1vlDwedZUvYa...,https://api.spotify.com/v1/audio-analysis/1vlD...,247920,4,1.0
2,spotify:track:1vlDwedZUvYaxt2FXoYGaJ,Song For Our Daughter,7B2edU3Q7btJoNsoHCNohM,Laura Marling,Held Down,https://open.spotify.com/track/1vlDwedZUvYaxt2...,56,2020-09-25T13:10:04Z,2020-04-10,2020,...,0.353,0.399,79.442,audio_features,1vlDwedZUvYaxt2FXoYGaJ,https://api.spotify.com/v1/tracks/1vlDwedZUvYa...,https://api.spotify.com/v1/audio-analysis/1vlD...,247920,4,1.0
3,spotify:track:1vlDwedZUvYaxt2FXoYGaJ,Song For Our Daughter,7B2edU3Q7btJoNsoHCNohM,Laura Marling,Held Down,https://open.spotify.com/track/1vlDwedZUvYaxt2...,56,2020-09-25T13:10:04Z,2020-04-10,2020,...,0.353,0.399,79.442,audio_features,1vlDwedZUvYaxt2FXoYGaJ,https://api.spotify.com/v1/tracks/1vlDwedZUvYa...,https://api.spotify.com/v1/audio-analysis/1vlD...,247920,4,11.0
4,spotify:track:1vlDwedZUvYaxt2FXoYGaJ,Song For Our Daughter,7B2edU3Q7btJoNsoHCNohM,Laura Marling,Held Down,https://open.spotify.com/track/1vlDwedZUvYaxt2...,56,2020-09-25T13:10:04Z,2020-04-10,2020,...,0.353,0.399,79.442,audio_features,1vlDwedZUvYaxt2FXoYGaJ,https://api.spotify.com/v1/tracks/1vlDwedZUvYa...,https://api.spotify.com/v1/audio-analysis/1vlD...,247920,4,63.0


Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:03gKdAQAswBo7v4p8F1An4,You Forgot It In People,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Pacific Theme,https://open.spotify.com/track/03gKdAQAswBo7v4...,46,2020-06-13T21:26:33Z,2003-03-11,2003,0.723,...,0.0979,0.767,129.725,audio_features,03gKdAQAswBo7v4p8F1An4,https://api.spotify.com/v1/tracks/03gKdAQAswBo...,https://api.spotify.com/v1/audio-analysis/03gK...,309187,4,509.0
spotify:track:0ey9VL7lFerteaCVvahKYV,Forgiveness Rock Record,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Sweetest Kill,https://open.spotify.com/track/0ey9VL7lFerteaC...,52,2020-03-12T22:40:03Z,2010-05-04,2010,0.709,...,0.0961,0.691,87.961,audio_features,0ey9VL7lFerteaCVvahKYV,https://api.spotify.com/v1/tracks/0ey9VL7lFert...,https://api.spotify.com/v1/audio-analysis/0ey9...,309307,4,509.0
spotify:track:2NVaKeIAUZwN5tXnn12EMi,Fate,4mLJ3XfOM5FPjSAWdQ2Jk7,Dr. Dog,The Breeze,https://open.spotify.com/track/2NVaKeIAUZwN5tX...,58,2019-11-03T17:46:14Z,2008-07-22,2008,0.611,...,0.6120,0.319,90.124,audio_features,2NVaKeIAUZwN5tXnn12EMi,https://api.spotify.com/v1/tracks/2NVaKeIAUZwN...,https://api.spotify.com/v1/audio-analysis/2NVa...,239093,4,463.0
spotify:track:7ywp9MWERblgkSMoMBugC0,Holiday,6RWjTQqILL7a1tQ0VapyLK,The Magnetic Fields,Strange Powers,https://open.spotify.com/track/7ywp9MWERblgkSM...,50,2019-11-03T17:42:52Z,1994,1994,0.668,...,0.1290,0.963,131.767,audio_features,7ywp9MWERblgkSMoMBugC0,https://api.spotify.com/v1/tracks/7ywp9MWERblg...,https://api.spotify.com/v1/audio-analysis/7ywp...,161613,4,455.0
spotify:track:7LZN7FkxHZk6maiN6NdI2i,Sunlit Youth,75dQReiBOHN37fQgWQrIAJ,Local Natives,Dark Days,https://open.spotify.com/track/7LZN7FkxHZk6mai...,63,2020-03-14T18:53:56Z,2016-09-09,2016,0.604,...,0.1250,0.590,115.038,audio_features,7LZN7FkxHZk6maiN6NdI2i,https://api.spotify.com/v1/tracks/7LZN7FkxHZk6...,https://api.spotify.com/v1/audio-analysis/7LZN...,180413,4,453.0


In [11]:
print('****PART 2 of 4: Get Disliked Songs Dataframe (if you have a Dislikes playlist)****')
try:
    define_scope()
    raw_disliked_song_list = get_raw_disliked_song_list()
    disliked_song_df = song_metadata_to_df(raw_disliked_song_list)
    disliked_song_df = add_audio_feats(disliked_song_df)
    genre_df = get_genres(disliked_song_df)
    genre_exploded_df = explode_genres(genre_df)
    disliked_song_df = add_genres(disliked_song_df, genre_exploded_df)
    disliked_genre_count_df = get_genre_counts(genre_exploded_df)
    disliked_genre_count_df['genre count'] = disliked_genre_count_df['genre count'] * -1
    disliked_song_df = establish_genre_score(disliked_genre_count_df, disliked_song_df)
except:
    pass

****PART 2 of 4: Get Disliked Songs Dataframe (if you have a Dislikes playlist)****
Logged in as 1254636534


,playlist_uri,song_count
playlist,,
"Jams 9/26, 8:35pm",spotify:playlist:6WFIRi0vyvxn8ZGzndekEa,25
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,454
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


,playlist_uri,song_count
playlist,,
"Jams 9/26, 8:35pm",spotify:playlist:6WFIRi0vyvxn8ZGzndekEa,25
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,454
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


Getting raw disliked song list
Transforming song raw data list into DataFrame(df) - this is basically a python version of an excel spreadsheet
Logged in as 1254636534
0/454 parsed
100/454 parsed
200/454 parsed
300/454 parsed
400/454 parsed


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012
spotify:track:6muOWhMh7Tf0UFUtMDBhUR,Orphans / Arabesque,4gzpq5DPGxSnKTe4SA8HAU,Coldplay,Orphans,https://open.spotify.com/track/6muOWhMh7Tf0UFU...,67,2019-11-06T02:40:45Z,2019-10-24,2019
spotify:track:5jafMI8FLibnjkYTZ33m0c,The Bends,4Z8W4fKeB5YxbusRsdQVPb,Radiohead,High And Dry,https://open.spotify.com/track/5jafMI8FLibnjkY...,73,2019-11-06T02:45:04Z,1995-03-28,1995
spotify:track:04KJ4NGb2T22y26sTO41Q0,Rise Of An Empire (Deluxe Edition),5OrB6Jhhrl9y2PK0pSV4VP,Young Money,Moment,https://open.spotify.com/track/04KJ4NGb2T22y26...,39,2019-11-06T02:59:28Z,2014-01-01,2014
spotify:track:6sICylTFKeiplGHltWnJ1k,Z,43O3c6wewpzPKwVaGEEtBM,My Morning Jacket,What A Wonderful Man,https://open.spotify.com/track/6sICylTFKeiplGH...,33,2019-11-06T03:02:29Z,2005-10-04,2005


...

,genre,genre count
0,dance pop,44
1,pop,37
2,pop rap,34
3,rock,34
4,alternative rock,30


Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre count
0,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-44.0
1,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-22.0
2,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-37.0
3,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-34.0
4,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-15.0


Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:00nQhTtc8FxgmB0NWypSE0,In Good Company,62W2NNKgzSx7GFJDUbJD6L,Java Gospel,You Rock My World,https://open.spotify.com/track/00nQhTtc8FxgmB0...,0,2020-04-02T13:32:30Z,2007-07-11,2007,0.662,...,0.3330,0.915,121.961,audio_features,00nQhTtc8FxgmB0NWypSE0,https://api.spotify.com/v1/tracks/00nQhTtc8Fxg...,https://api.spotify.com/v1/audio-analysis/00nQ...,245947,4,0.0
spotify:track:1TKRumlRMXv2SEKPhueya0,The Step Daddy (Explicit),6RhzdrHfuZEMnZTNRjbTpz,Hitman Sammy Sam,Step Daddy,https://open.spotify.com/track/1TKRumlRMXv2SEK...,24,2020-04-02T13:23:41Z,2003-01-01,2003,0.930,...,0.0767,0.348,118.997,audio_features,1TKRumlRMXv2SEKPhueya0,https://api.spotify.com/v1/tracks/1TKRumlRMXv2...,https://api.spotify.com/v1/audio-analysis/1TKR...,230493,4,0.0
spotify:track:6QeMRyd3BmSfJbaNay3dpe,R&B - 100 Hits - The Greatest R n B album - 10...,2mxe0TnaNL039ysAj51xPQ,Various Artists,The World's Greatest,https://open.spotify.com/track/6QeMRyd3BmSfJba...,53,2020-04-25T23:19:22Z,2013-06-21,2013,0.661,...,0.0469,0.721,95.971,audio_features,6QeMRyd3BmSfJbaNay3dpe,https://api.spotify.com/v1/tracks/6QeMRyd3BmSf...,https://api.spotify.com/v1/audio-analysis/6QeM...,277600,4,0.0
spotify:track:5nB5bek72WORC5jegCIPNk,Untitled,2mxe0TnaNL039ysAj51xPQ,R. Kelly,Number One,https://open.spotify.com/track/5nB5bek72WORC5j...,40,2020-04-02T16:30:51Z,2009-11-27,2009,0.650,...,0.1800,0.576,136.796,audio_features,5nB5bek72WORC5jegCIPNk,https://api.spotify.com/v1/tracks/5nB5bek72WOR...,https://api.spotify.com/v1/audio-analysis/5nB5...,260867,4,0.0
spotify:track:2x5C1qAOJ1KJFqM6uhOsgT,Blocked Numbers,2GdgDnUG0ZMFZiu2122pGQ,Crystal Skulls,Hussy,https://open.spotify.com/track/2x5C1qAOJ1KJFqM...,16,2020-06-29T13:48:04Z,2005-01-01,2005,0.660,...,0.1290,0.792,119.568,audio_features,2x5C1qAOJ1KJFqM6uhOsgT,https://api.spotify.com/v1/tracks/2x5C1qAOJ1KJ...,https://api.spotify.com/v1/audio-analysis/2x5C...,279093,4,0.0


In [12]:
print('')
print('****PART 3 of 4: Create "my_songs" df using liked (and optionally disliked) songs***')
try:
    my_songs_df = combine_dfs_to_create_my_songs_df(liked_song_df,disliked_song_df,'.')
    genre_count_df = pd.concat([liked_genre_count_df, disliked_genre_count_df])
except:
    my_songs_df = liked_song_df
    genre_count_df = liked_genre_count_df
#my_songs_df = pd.read_pickle("./my_songs.pkl")


****PART 3 of 4: Create "my_songs" df using liked (and optionally disliked) songs***
Combining dfs to create my_songs_df


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score,Liked
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:03gKdAQAswBo7v4p8F1An4,You Forgot It In People,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Pacific Theme,https://open.spotify.com/track/03gKdAQAswBo7v4...,46,2020-06-13T21:26:33Z,2003-03-11,2003,0.723,...,0.767,129.725,audio_features,03gKdAQAswBo7v4p8F1An4,https://api.spotify.com/v1/tracks/03gKdAQAswBo...,https://api.spotify.com/v1/audio-analysis/03gK...,309187,4,509.0,1
spotify:track:0ey9VL7lFerteaCVvahKYV,Forgiveness Rock Record,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Sweetest Kill,https://open.spotify.com/track/0ey9VL7lFerteaC...,52,2020-03-12T22:40:03Z,2010-05-04,2010,0.709,...,0.691,87.961,audio_features,0ey9VL7lFerteaCVvahKYV,https://api.spotify.com/v1/tracks/0ey9VL7lFert...,https://api.spotify.com/v1/audio-analysis/0ey9...,309307,4,509.0,1
spotify:track:2NVaKeIAUZwN5tXnn12EMi,Fate,4mLJ3XfOM5FPjSAWdQ2Jk7,Dr. Dog,The Breeze,https://open.spotify.com/track/2NVaKeIAUZwN5tX...,58,2019-11-03T17:46:14Z,2008-07-22,2008,0.611,...,0.319,90.124,audio_features,2NVaKeIAUZwN5tXnn12EMi,https://api.spotify.com/v1/tracks/2NVaKeIAUZwN...,https://api.spotify.com/v1/audio-analysis/2NVa...,239093,4,463.0,1
spotify:track:7ywp9MWERblgkSMoMBugC0,Holiday,6RWjTQqILL7a1tQ0VapyLK,The Magnetic Fields,Strange Powers,https://open.spotify.com/track/7ywp9MWERblgkSM...,50,2019-11-03T17:42:52Z,1994,1994,0.668,...,0.963,131.767,audio_features,7ywp9MWERblgkSMoMBugC0,https://api.spotify.com/v1/tracks/7ywp9MWERblg...,https://api.spotify.com/v1/audio-analysis/7ywp...,161613,4,455.0,1
spotify:track:7LZN7FkxHZk6maiN6NdI2i,Sunlit Youth,75dQReiBOHN37fQgWQrIAJ,Local Natives,Dark Days,https://open.spotify.com/track/7LZN7FkxHZk6mai...,63,2020-03-14T18:53:56Z,2016-09-09,2016,0.604,...,0.590,115.038,audio_features,7LZN7FkxHZk6maiN6NdI2i,https://api.spotify.com/v1/tracks/7LZN7FkxHZk6...,https://api.spotify.com/v1/audio-analysis/7LZN...,180413,4,453.0,1


In [13]:
print('****PART 4 of 4: Get songs from featured playlists and score them based on genre****')
print(f"Creating featured playlist songs for: {(pd.Timestamp.now()-timedelta(days=0*365)).strftime('%Y-%m-%dT%H:%M:%S.%Z')}")
playlist_df = get_featured_playlist_uris('US',(pd.Timestamp.now()-timedelta(days=0*365)).strftime('%Y-%m-%dT%H:%M:%S.%Z'))
# print(f"{display(playlist_df)}")
raw_featured_playlist_songs_list = get_raw_featured_playlist_song_list(playlist_df)
featured_playlist_song_df = song_metadata_to_df(raw_featured_playlist_songs_list)
#featured_playlist_song_df = add_audio_feats(featured_playlist_song_df)
genre_df = get_genres(featured_playlist_song_df)
genre_exploded_df = explode_genres(genre_df)
featured_playlist_song_df = add_genres(featured_playlist_song_df, genre_exploded_df)
featured_playlist_song_df = establish_genre_score(genre_count_df, featured_playlist_song_df)
create_playlist(pred_like_playlist_name, featured_playlist_song_df)
print('****Complete! Check Spotify to see if playlist was filled with songs.****')

****PART 4 of 4: Get songs from featured playlists and score them based on genre****
Creating featured playlist songs for: 2020-09-26T21:43:33.
Getting raw_featured_playlist_df


,playlist_uri,song_count
playlist,,
Dance Pop,spotify:playlist:37i9dQZF1DWZQaaqNMbbXa,116
Throwback Jams,spotify:playlist:37i9dQZF1DX8ky12eWIvcW,93
Most Necessary,spotify:playlist:37i9dQZF1DX2RxBh64BHjQ,100
Latin Party Anthems,spotify:playlist:37i9dQZF1DWVcbzTgVpNRm,100
Young Nashville,spotify:playlist:37i9dQZF1DWTvM9hRJYH10,66
Mind Right,spotify:playlist:37i9dQZF1DX5hR0J49CmXC,50
Mega Hit Mix,spotify:playlist:37i9dQZF1DXbYM3nMM0oPk,75
Ultimate Indie,spotify:playlist:37i9dQZF1DX2Nc3B70tvx0,102
Soft Pop Hits,spotify:playlist:37i9dQZF1DWTwnEm1IYyoj,234


Transforming song raw data list into DataFrame(df) - this is basically a python version of an excel spreadsheet
Logged in as 1254636534
0/850 parsed
100/850 parsed
200/850 parsed
300/850 parsed
400/850 parsed
500/850 parsed
600/850 parsed
700/850 parsed
800/850 parsed


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:2IDHoSlbG0CKH9rhq68wTb,Say My Name Remixes,21mKp7DqtSNHhCAU2ugvUw,ODESZA,Say My Name (feat. Zyra) - RAC Mix,https://open.spotify.com/track/2IDHoSlbG0CKH9r...,44,2020-07-24T04:24:28Z,2014-10-13,2014
spotify:track:24ySl2hOPGCDcxBxFIqWBu,Rain On Me (with Ariana Grande),1HY2Jd0NmPuamShAr6KMms,Lady Gaga,Rain On Me (with Ariana Grande),https://open.spotify.com/track/24ySl2hOPGCDcxB...,89,2020-07-24T04:24:28Z,2020-05-22,2020
spotify:track:24Yi9hE78yPEbZ4kxyoXAI,Roses (Imanbek Remix),0H39MdGGX6dbnnQPt6NQkZ,SAINt JHN,Roses - Imanbek Remix,https://open.spotify.com/track/24Yi9hE78yPEbZ4...,93,2020-07-24T04:24:28Z,2019-10-09,2019
spotify:track:7C7yqFTM0ncyJ04GIKrxdV,Halcyon,0X2BH1fck6amBIoJhDVmmJ,Ellie Goulding,Anything Could Happen,https://open.spotify.com/track/7C7yqFTM0ncyJ04...,57,2020-07-24T04:24:28Z,2012-01-01,2012
spotify:track:57YMEPQhEy5F5Q715mW0n8,Surrender (Martin Jensen Remix),2WxjxdeF7GGdcCK276qViY,Natalie Taylor,Surrender - Martin Jensen Remix,https://open.spotify.com/track/57YMEPQhEy5F5Q7...,65,2020-07-24T04:24:28Z,2020-05-22,2020


Getting genres ...
423 distinct artists
0/423 parsed
0:ODESZA:['chillwave', 'ninja']
1:Lady Gaga:['dance pop', 'pop']
2:SAINt JHN:['melodic rap', 'pop rap', 'rap']
3:Ellie Goulding:['dance pop', 'indietronica', 'metropopolis', 'pop', 'post-teen pop', 'uk pop']
4:Natalie Taylor:[]
5:Joel Corry:['dance pop', 'house', 'pop', 'tropical house', 'uk dance']
6:Zedd:['complextro', 'dance pop', 'edm', 'electro house', 'electropop', 'german techno', 'pop', 'post-teen pop', 'tropical house']
7:A R I Z O N A:['edm', 'electropop', 'indie poptimism', 'pop', 'tropical house']
8:Carly Rae Jepsen:['canadian pop', 'dance pop', 'electropop', 'indie poptimism', 'pop', 'post-teen pop']
9:Kygo:['edm', 'pop', 'tropical house']
10:ayokay:['electropop', 'indie electropop', 'indie poptimism', 'pop', 'pop edm', 'pop rap', 'tropical house']
11:Major Lazer:['dance pop', 'edm', 'electro house', 'moombahton', 'pop', 'pop rap']
12:Sofi Tukker:['electropop']
13:Sam Feldt:['dance pop', 'edm', 'electro house', 'pop', 'p

...

Logged in as 1254636534


,playlist_uri,song_count
playlist,,
90s,spotify:playlist:1g2dqbzbKfn4I7RfggDGHW,0
"Jams 9/26, 8:35pm",spotify:playlist:6WFIRi0vyvxn8ZGzndekEa,25
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,454
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


,playlist_uri,song_count
playlist,,
90s,spotify:playlist:1g2dqbzbKfn4I7RfggDGHW,0
"Jams 9/26, 8:35pm",spotify:playlist:6WFIRi0vyvxn8ZGzndekEa,25
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,454
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,genre score,pct_rank
uri,,,,,,,,,,,
spotify:track:5XRfIAdgYxZ3jLiqxSZnRj,Whole New Mess,6mKqFxGMS5TGDZI3XkT5Rt,Angel Olsen,Whole New Mess,https://open.spotify.com/track/5XRfIAdgYxZ3jLi...,53,2020-08-07T04:01:00Z,2020-07-28,2020,544.0,1.000000
spotify:track:3pYB28IRzhtR5cHXLINchp,Shore,4EVpmkEwrLYEg6jIsiPMIb,Fleet Foxes,Can I Believe You,https://open.spotify.com/track/3pYB28IRzhtR5cH...,62,2020-09-25T04:01:00Z,2020-09-22,2020,534.0,0.998182
spotify:track:3Hs0xJpEOcJsWeayaLbeXR,Ferris Wheel,39vA9YljbnOApXKniLWBZv,Sylvan Esso,Ferris Wheel,https://open.spotify.com/track/3Hs0xJpEOcJsWea...,64,2020-08-07T04:01:00Z,2020-07-21,2020,390.0,0.995455
spotify:track:69TJLfGTUYSugs2Yi1dnG3,Rooftop Dancing,39vA9YljbnOApXKniLWBZv,Sylvan Esso,Rooftop Dancing,https://open.spotify.com/track/69TJLfGTUYSugs2...,60,2020-09-25T04:01:00Z,2020-08-11,2020,390.0,0.995455
spotify:track:4tZwZVNhVCeBQYQtgQN5vW,"Identical (From The Motion Picture ""On The Roc...",1xU878Z1QtBldR7ru9owdU,Phoenix,"Identical - From The Motion Picture ""On The Ro...",https://open.spotify.com/track/4tZwZVNhVCeBQYQ...,63,2020-09-04T04:02:00Z,2020-08-19,2020,378.0,0.992727


****Complete! Check Spotify to see if playlist was filled with songs.****


In [ ]:
################## END OF OFFICIAL SCRIPT - BELOW IS JUST PLAYING AROUND ###############